Checking if there is a GPU device available for training. I did the training on Kaggle, so this is just to debug.

In [ ]:
import torch

print("CUDA Available:", torch.cuda.is_available())
print("Device Name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU detected")


In [ ]:
from torch import nn
from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader
import os
from PIL import Image
from d2l import torch as d2l
import os
import pandas as pd

## Below I define:
- A custom Test Dataset class to make the test dataloader
- The network architecture (to be changed and modified further)
- A custom Data class to be passed into the trainer.fit() method later

In [ ]:

class TestDataset(Dataset):
    def __init__(self, root, transform=None):
        self.root = root
        self.transform = transform
        self.image_paths = [
            os.path.join(root, fname)
            for fname in os.listdir(root)
            if fname.endswith(('jpg', 'png', 'jpeg'))
        ]

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, img_path

def init_cnn(module):
    if isinstance(module, nn.Conv2d) or isinstance(module, nn.Linear):
        nn.init.xavier_uniform_(module.weight)

class NandNet(d2l.Classifier):

    def __init__(self, numchannels, lr=0.1, num_classes=2):
        super().__init__()
        self.save_hyperparameters()
        self.net = nn.Sequential(
            nn.LazyConv2d(numchannels, kernel_size=3, stride=1, padding=1),
            nn.LazyBatchNorm2d(), nn.ReLU(),
            nn.LazyConv2d(numchannels, kernel_size=3, stride=1, padding=1),
            nn.LazyBatchNorm2d(), nn.ReLU(),
            nn.LazyConv2d(numchannels, kernel_size=3, stride=1, padding=1),
            nn.LazyBatchNorm2d(), nn.ReLU(),
            nn.Flatten(), nn.LazyLinear(64), nn.LazyBatchNorm1d(),
            nn.ReLU(), nn.LazyLinear(16), nn.LazyBatchNorm1d(),
            nn.ReLU(), nn.LazyLinear(num_classes)
            )

class MyData(d2l.DataModule):
    def __init__(self, batch_size=64):
        super().__init__()
        self.save_hyperparameters()
        self.train = trainset
        self.val = valset
        self.batch_size = batch_size
        

    def text_labels(self, indices):
        labels = ["yes", "no"]
        return [labels[int(i)] for i in indices]

    def get_dataloader(self, train):
        data = self.train if train else self.val
        return DataLoader(data, self.batch_size, shuffle=train, num_workers=16)

    def visualize(self, batch, nrows=1, ncols=8, labels=None):
        if labels is None:
            labels = []
        X, y = batch
        if not labels:
            labels = self.text_labels(y)
        d2l.show_images(X.squeeze(1), nrows, ncols, titles=labels)


## Below I define:
- The transformations to apply to our data in order to augment it
- THe variables that hold the training, validation, and test sets

In [ ]:
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(10),
    transforms.RandomResizedCrop(
        size=224,
        scale=(0.85, 1.15),
        ratio=(0.75, 1.33)
    ),
    transforms.ToTensor(),
])

trainset = datasets.ImageFolder(
    root = 'dl2425_challenge_dataset/train',
    transform = transform
)
valset = datasets.ImageFolder(
    root = 'dl2425_challenge_dataset/val',
    transform = transform
)
testset = TestDataset(
    root = 'dl2425_challenge_dataset/test',
    transform = transform
)

testloader = DataLoader(testset, batch_size=64, shuffle=False)

## Below I define:
- The code to train the model
- (If ran in a Jupyter Notebook cell, it also outputs a dynamic train-acc graph)

In [ ]:
"""if __name__ == '__main__':
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = NandNet(numchannels=6, lr=0.1, num_classes=2)
    model = model.to(device)
    trainer = d2l.Trainer(max_epochs=10, num_gpus=1)
    dummy_input = torch.randn(64, 3, 224, 224).to(device)
    model(dummy_input)
    model.apply(init_cnn)
    data = MyData(batch_size=64)
    trainer.fit(model, data)
    torch.save(model.state_dict(), 'model_weights.pth')"""

## Below I define:
- The model that is to be loaded from the models directory
- The test loop
- The results Data Frame which takes multiple dictionaries in the form {id: class} and converts them into cells in a .csv file
- Saves the predictions.csv file to desired path (this is what we will submit)

In [ ]:
device = torch.device("cuda")
model = NandNet(numchannels=6, lr=0.1, num_classes=2)
current_model = 'models/model_weights_3layers_simple.pth'
model.load_state_dict(torch.load(current_model))
model = model.to(device)
model.eval()

results = []

with torch.no_grad():
    for images, paths in testloader:
        images = images.to(device)
        outputs = model(images)
        
        probabilities = torch.softmax(outputs, dim=1)
        predictions = torch.argmax(probabilities, dim=1) 

        for path, pred in zip(paths, predictions.cpu().numpy()):
            filename = os.path.basename(path) 
            results.append({"id": filename, "class": int(pred)})

# save model predictions!!
results = sorted(results, key = lambda x: int(x["id"].replace(".jpg", "")))
df = pd.DataFrame(results)
df.to_csv('predictions.csv', index=False)
print("Predictions saved to 'predictions.csv'")

## Next steps:
- Make the model better
- Utilize as few premade d2l methods as possible
- Once we are confident in our prediction results, merge (labeled) test, train, and val
- Train the final model on all of it